In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import glob
import imageio
from keras.models import Model
from keras.models import load_model
from keras.layers import *
from keras import backend as K
from keras.optimizers import Adam
from keras.utils import to_categorical
from src.data.dataset import FergZeroOne, Ferg
from keras.callbacks import ModelCheckpoint
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


In [3]:
ferg = Ferg.from_hdf5(select_people=[0, 1, 2, 3, 4, 5], transform=True)
train_data = ferg.get_train()
test_data = ferg.get_test()
for key, value in train_data.items():
    print(f'{key}: {value.shape}')

for key, value in test_data.items():
    print(f'{key}: {value.shape}')

x_train, y_train, p_train = train_data['x'], train_data['y'], train_data['p']
x_test, y_test, p_test = test_data['x'], test_data['y'], test_data['p']

x: (47401, 64, 64, 3)
y: (47401, 7)
p: (47401, 6)
x: (8365, 64, 64, 3)
y: (8365, 7)
p: (8365, 6)


In [4]:
# train a classify model aiming to classify face expression(y)
img_dim = 64
num_cnn = 4
z_dim = 64
y_class = 7
p_class = 6
x_in = Input(shape=(img_dim, img_dim, 3))
x = x_in
all_models = []
all_models.append(Model(x_in, x))
for i in range(num_cnn):
    x = Conv2D(z_dim,
               kernel_size=(3,3),
               padding='SAME')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    all_models.append(Model(x_in, x))
x = GlobalMaxPooling2D()(x)
all_models.append(Model(x_in, x))
x = Dense(z_dim, activation='relu')(x)
all_models.append(Model(x_in, x))
x = Dense(y_class, activation='softmax')(x)
all_models.append(Model(x_in, x))
model = Model(x_in, x)
model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 64)        0         
__________

In [ ]:
callback = ModelCheckpoint('./classify_task_best_weights.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test), callbacks=[callback])

Train on 47401 samples, validate on 8365 samples
Epoch 1/5
46720/47401 [============================>.] - ETA: 0s - loss: 0.1147 - acc: 0.9684

In [ ]:
model.load_weights('./classify_task_best_weights.h5')

In [ ]:
def evaluate(x_train, x_test, p_train, p_test, num_classes=6):
    input_shape = x_train.shape[1:]
    x_in = Input(shape=input_shape)
    x = x_in
    if len(input_shape) == 1:
        #64 or 7
        dim = input_shape[0]
        x = Dense(dim, activation='relu')(x)
    else:
        #(64, 64, 64) or (64, 64, 3)
        z_dim = input_shape[0]
        for i in range(num_cnn):
            x = Conv2D(z_dim,
                       kernel_size=(3,3),
                       padding='SAME')(x)
            x = BatchNormalization()(x)
            x = LeakyReLU(0.2)(x)
        x = GlobalMaxPooling2D()(x)
    x = Dense(num_classes, activation='relu')(x)
    eva_model = Model(x_in, x)
    eva_model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['acc'])
    history = eva_model.fit(x_train, p_train, epochs=5, batch_size=64, validation_data=(x_test, p_test))
    return np.max(history.history['val_acc'])

all_acc = []
for i, model in enumerate(all_models):
    if i == 0:
        continue
    x_eva_train = model.predict(x_train, batch_size=64)
    x_eva_test = model.predict(x_test, batch_size=64)
    all_acc.append(evaluate(x_eva_train, x_eva_test, p_train, p_test))
    
print(all_acc)